In [9]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

from matplotlib import cm, colors
import matplotlib.pyplot as plt

import numpy as np

import dataUtils as du

import tempfile
import os

def GetData(filename, train_frac=0.5, branches=['pileup', 'amp', 'phase', 'trace'],  treename="OutputTree"):
    '''
    Returns TFile as a pandas dataframe
    '''
    try: 
        import uproot
        import awkward as ak
        import numpy as np
        from sklearn.model_selection import train_test_split
        
    except ImportError:
        print("Please install uproot, awkward, numpy, and pandas")
        return
  
    file = uproot.open(filename)
    tree = file[treename]
    data = {}
    for branch in branches:
        data[branch] = ak.to_numpy(tree[branch].arrays()[branch])
    
    train_size = int(len(data["trace"])*train_frac)
    test_size = len(data["trace"]) - train_size
    print("Train size: ",train_size)
    print("Test size: ",test_size)

    x = np.array(data["trace"])
    y_pileup = np.array(data["pileup"])
    y_amp = np.array(data["amp"])
    y_phase = np.array(data["phase"])

    x_train, x_test, y_pileup_train, y_pileup_test, y_amp_train, y_amp_test, y_phase_train, y_phase_test = train_test_split(x, y_pileup, y_amp, y_phase, test_size=test_size)

    return ((x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test))

(x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test) = GetData("../data/Data10percFloat.root")

Train size:  298980
Test size:  298981


In [2]:
AUGMENTATION = 8
TRACELENGTH = 250
INPUTSIZE = TRACELENGTH-4*AUGMENTATION
print("Input size: ", INPUTSIZE)

Input size:  218


In [21]:
import json 

AUGMENTATION = 8
TRACELENGTH = 250
INPUTSIZE = TRACELENGTH-4*AUGMENTATION
VALIDATION_SPLIT = 0.2

TRAIN_PERCENT = 0.1
DATAFILE = "/home/tmengel/ML4PileUp/data/Data10percFloat.root"
TREENAME = "OutputTree"
OUTPUT_DIR = "/home/tmengel/ML4PileUp/network-pruning/"
EPOCHS_SEARCH = 10
EPOCHS_FINAL = 100
BATCH_SIZE = 32

file = "hyper_config.json"

ModelKeys = {  
    "AUGMENTATION" : AUGMENTATION,
    "TRACELENGTH" : TRACELENGTH,
    "INPUTSIZE" : INPUTSIZE,
    "VALIDATION_SPLIT" : VALIDATION_SPLIT,
    "TRAIN_PERCENT" : TRAIN_PERCENT,
    "DATAFILE" : DATAFILE,
    "TREENAME" : TREENAME,
    "EPOCHS_SEARCH" : EPOCHS_SEARCH,
    "EPOCHS_FINAL" : EPOCHS_FINAL,
    "BATCH_SIZE" : BATCH_SIZE,
    "DATA_FILE" : DATAFILE,
    "OUTPUT_DIR" : OUTPUT_DIR,
    "MODEL_TYPES" : ["Amp", "Pileup", "Phase"],
        "Amp": {
                "LOSS" : "mse",
                "OUTPUT_ACVTIVATIONS" : ['relu', 'linear'],
                "OBJECTIVE" : "val_loss",
                "EARLY_STOPPING_VAR" : "val_loss",
                "EARLY_STOPPING_MODE" : "min",
                "DIRECTORY" : "AmpDir",
                "PROJECT_NAME" : "AmpHyperTune",
                "TUNED_MODEL" : "AmpHyperTunedModel"
                },
        "Pileup" : {
                "LOSS" : "bce",
                "OUTPUT_ACVTIVATIONS" : ['sigmoid', 'tanh'],
                "OBJECTIVE" : "val_accuracy",
                "EARLY_STOPPING_VAR" : "val_accuracy",
                "EARLY_STOPPING_MODE" : "max",
                "DIRECTORY" : "PileupDir",
                "PROJECT_NAME" : "PileupHyperTune",
                "TUNED_MODEL" : "PileupHyperTunedModel"
                },
        "Phase" : {
                "LOSS" : "mse",
                "OUTPUT_ACVTIVATIONS" : ['relu', 'linear'],
                "OBJECTIVE" : "val_loss",
                "EARLY_STOPPING_VAR" : "val_loss",
                "EARLY_STOPPING_MODE" : "min",
                "DIRECTORY" : "PhaseDir",
                "PROJECT_NAME" : "PhaseHyperTune",
                "TUNED_MODEL" : "PhaseHyperTunedModel"
                }
}

with open(file, "w") as f:
    json.dump(ModelKeys, f, indent=2)



In [19]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

import sys

ModelKeys = {  
      "Amp": {
            "LOSS" : "mse",
            "OUTPUT_ACVTIVATIONS" : ['relu', 'linear'],
            "OBJECTIVE" : "val_loss",
            "EARLY_STOPPING_VAR" : "val_loss",
            "EARLY_STOPPING_MODE" : "min",
            "DIRECTORY" : "AmpDir",
            "PROJECT_NAME" : "AmpHyperTune",
            "TUNED_MODEL" : "AmpHyperTunedModel"
            },
    "Pileup" : {
            "LOSS" : "bce",
            "OUTPUT_ACVTIVATIONS" : ['sigmoid', 'tanh'],
            "OBJECTIVE" : "val_accuracy",
            "EARLY_STOPPING_VAR" : "val_accuracy",
            "EARLY_STOPPING_MODE" : "max",
            "DIRECTORY" : "PileupDir",
            "PROJECT_NAME" : "PileupHyperTune",
            "TUNED_MODEL" : "PileupHyperTunedModel"
            },
    "Phase" : {
            "LOSS" : "mse",
            "OUTPUT_ACVTIVATIONS" : ['relu', 'linear'],
            "OBJECTIVE" : "val_loss",
            "EARLY_STOPPING_VAR" : "val_loss",
            "EARLY_STOPPING_MODE" : "min",
            "DIRECTORY" : "PhaseDir",
            "PROJECT_NAME" : "PhaseHyperTune",
            "TUNED_MODEL" : "PhaseHyperTunedModel"
            }
}

# constants
AUGMENTATION = 8
TRACELENGTH = 250
INPUTSIZE = TRACELENGTH-4*AUGMENTATION
VALIDATION_SPLIT = 0.2

TRAIN_PERCENT = 0.1
DATAFILE = "../data/Data10percFloat.root"

EPOCHS_SEARCH = 10
EPOCHS_FINAL = 100
BATCH_SIZE = 32



# Model specific constants
TYPE = "Amp"
VALID_TYPES = ModelKeys.keys()
if TYPE not in VALID_TYPES:
    print("Invalid model type")
    sys.exit(1)

LOSS = ModelKeys[TYPE]["LOSS"]

# get current directory
DIRECTORY = f'{os.getcwd()}/{ModelKeys[TYPE]["DIRECTORY"]}'
PROJECT_NAME = ModelKeys[TYPE]["PROJECT_NAME"]

OUTPUT_ACVTIVATIONS = ModelKeys[TYPE]["OUTPUT_ACVTIVATIONS"]
OBJECTIVE = ModelKeys[TYPE]["OBJECTIVE"]
TUNED_MODEL = ModelKeys[TYPE]["TUNED_MODEL"]

EARLY_STOPPING_VAR = ModelKeys[TYPE]["EARLY_STOPPING_VAR"]
EARLY_STOPPING_MODE = ModelKeys[TYPE]["EARLY_STOPPING_MODE"]

# print out info
print("====================================")
print("Constants\n")
print("AUGMENTATION: ", AUGMENTATION)
print("TRACELENGTH: ", TRACELENGTH)
print("INPUTSIZE: ", INPUTSIZE)
print("VALIDATION_SPLIT: ", VALIDATION_SPLIT)
print("TRAIN_PERCENT: ", TRAIN_PERCENT)
print("DATAFILE: ", DATAFILE)
print("EPOCHS_SEARCH: ", EPOCHS_SEARCH)
print("EPOCHS_FINAL: ", EPOCHS_FINAL)
print("BATCH_SIZE: ", BATCH_SIZE)
print("====================================")
print("Model specific constants\n")
print("TYPE: ", TYPE)
print("LOSS: ", LOSS)
print("OUTPUT_ACVTIVATIONS: ", OUTPUT_ACVTIVATIONS)
print("OBJECTIVE: ", OBJECTIVE)
print("DIRECTORY: ", DIRECTORY)
print("PROJECT_NAME: ", PROJECT_NAME)
print("TUNED_MODEL: ", TUNED_MODEL)
print("EARLY_STOPPING_VAR: ", EARLY_STOPPING_VAR)
print("EARLY_STOPPING_MODE: ", EARLY_STOPPING_MODE)
print("====================================")




def GetData(filename, train_frac=0.5, branches=['pileup', 'amp', 'phase', 'trace'],  treename="OutputTree"):
    '''
    Returns TFile as a pandas dataframe
    '''
    try: 
        import uproot
        import awkward as ak
        import numpy as np
        from sklearn.model_selection import train_test_split
        
    except ImportError:
        print("Please install uproot, awkward, numpy, and pandas")
        return
  
    file = uproot.open(filename)
    tree = file[treename]
    data = {}
    for branch in branches:
        data[branch] = ak.to_numpy(tree[branch].arrays()[branch])
    
    train_size = int(len(data["trace"])*train_frac)
    test_size = len(data["trace"]) - train_size
    print("Train size: ",train_size)
    print("Test size: ",test_size)

    x = np.array(data["trace"])
    y_pileup = np.array(data["pileup"])
    y_amp = np.array(data["amp"])
    y_phase = np.array(data["phase"])

    x_train, x_test, y_pileup_train, y_pileup_test, y_amp_train, y_amp_test, y_phase_train, y_phase_test = train_test_split(x, y_pileup, y_amp, y_phase, test_size=test_size)

    return ((x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test))

def GetDataForModel(filename, model_type, train_frac=0.5,  treename="OutputTree"):
     
    # get the data
    (x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test) = GetData(filename, train_frac=train_frac, branches=['pileup', 'amp', 'phase', 'trace'], treename=treename)

    # get the model specific data
    if model_type == "Amp":
        y_train = y_amp_train
        y_test = y_amp_test
    elif model_type == "Pileup":
        y_train = y_pileup_train
        y_test = y_pileup_test
    elif model_type == "Phase":
        y_train = y_phase_train
        y_test = y_phase_test
    else:
        print("Invalid model type")
        return None
    
    return ((x_train, y_train), (x_test, y_test))

def ModelHyperTune(hp):
        
        model = keras.Sequential()

        # input layer
        kernel_size_input = hp.Int('kernel_size_0', min_value=2, max_value=18, step=4)
        filters_input = hp.Int('filters_0', min_value=16, max_value=128, step=16)
        activation_input = hp.Choice('activation_0', values=['tanh', 'relu'])

        model.add(keras.layers.Conv1D(kernel_size=kernel_size_input, 
                                      filters=filters_input, 
                                      activation=activation_input, 
                                      name='conv0', 
                                      input_shape=(INPUTSIZE,1)))

        # max pooling options
        max_pooling = hp.Boolean('max_pooling')
        pool_size = hp.Int('pool_size', min_value=2, max_value=8, step=2)
        # add max pooling
        if max_pooling:
                model.add(keras.layers.MaxPooling1D(pool_size=pool_size, name='max_pooling0'))        

        # drouput options
        dropout_conv = hp.Boolean('dropout_conv')       
        dropout_conv_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
        # add dropout
        if dropout_conv:
                model.add(keras.layers.Dropout(dropout_conv_rate, name='dropout_conv0'))
        
        # # hidden conv layers
        for i in range(hp.Int('num_conv_layers', min_value=0, max_value=3, step=1)):
                kernel_size = hp.Int(f'kernel_size_{i+1}', min_value=1, max_value=16, step=4)
                filters = hp.Int(f'filters_{i+1}', min_value=16, max_value=128, step=16)
                activation = hp.Choice(f'activation_{i+1}', values=['tanh', 'relu'])

                model.add(keras.layers.Conv1D(kernel_size=kernel_size, 
                                              filters=filters, 
                                              activation=activation, 
                                              name=f'conv{i+1}'))
                # add max pooling
                if max_pooling:
                        model.add(keras.layers.MaxPooling1D(pool_size=pool_size, name=f'max_pooling{i+1}'))

                # add dropout
                if dropout_conv:
                        model.add(keras.layers.Dropout(dropout_conv_rate, name=f'dropout_conv{i+1}'))
        
        # # flatten layer
        model.add(keras.layers.Flatten(name='flat0'))

        # initial dense layer
        units_dense = hp.Int('units_dense0', min_value=32, max_value=512, step=32)
        activation_dense = hp.Choice('activation_dense0', values=['tanh', 'relu'])
        model.add(keras.layers.Dense(units=units_dense, activation=activation_dense, name='dense0'))

        # # hidden dense layers
        dropout_dense = hp.Boolean('dropout_dense')
        dropout_dense_rate = hp.Float('dropout_rate_dense', min_value=0.1, max_value=0.5, step=0.1)

        if dropout_dense:
                model.add(keras.layers.Dropout(dropout_dense_rate, name='dropout_dense0'))

        # add hidden dense layers
        for i in range(hp.Int('num_dense_layers', min_value=0, max_value=3, step=1)):
                units = hp.Int(f'units_dense_{i+1}', min_value=32, max_value=512, step=32)
                activation = hp.Choice(f'activation_dense_{i+1}', values=['tanh', 'relu'])
                model.add(keras.layers.Dense(units=units, activation=activation, name=f'dense{i+1}'))

                if dropout_dense:
                        model.add(keras.layers.Dropout(dropout_dense_rate, name=f'dropout_dense{i+1}'))
                                  
        
        # # output layer
        avtivation_output = hp.Choice('activation_output', values=OUTPUT_ACVTIVATIONS)
        model.add(keras.layers.Dense(1, activation=avtivation_output, name='output'))

        # compile model
        lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                            loss=LOSS,
                            metrics=[LOSS, 'accuracy']
                            )
        
        return model

def PrintBestHyperparameters(best_hps):
    best_hps_dict = best_hps.values
    print("Best hyperparameters:")
    for key in best_hps_dict:
        print(key, best_hps_dict[key])


tuner = kt.Hyperband(ModelHyperTune,
                     objective=OBJECTIVE,
                     max_epochs=EPOCHS_SEARCH,
                     overwrite=True,
                     factor=3,
                     directory=str(DIRECTORY),
                     project_name=str(PROJECT_NAME)
                     )

stop_early = tf.keras.callbacks.EarlyStopping(
                                              monitor=EARLY_STOPPING_VAR,
                                              patience=5, 
                                              restore_best_weights=True, 
                                              min_delta=0.001, 
                                              mode=EARLY_STOPPING_MODE
                                            )



(x_train, y_train), (x_test, y_test) = GetDataForModel(filename= DATAFILE, 
                                                       model_type=TYPE, 
                                                       train_frac=TRAIN_PERCENT)

tuner.search(x_train, y_train, epochs=EPOCHS_SEARCH, validation_split=VALIDATION_SPLIT, callbacks=[stop_early])


best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
PrintBestHyperparameters(best_hps)

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train,
                    epochs=EPOCHS_FINAL,
                    validation_split=VALIDATION_SPLIT, 
                    callbacks=[stop_early])

val_obj_per_epoch = history.history[OBJECTIVE]
best_epoch = val_obj_per_epoch.index(max(val_obj_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

# Build the model with the optimal hyperparameters and train it on the data
hypermodel = tuner.hypermodel.build(best_hps)

# # Retrain the model
hypermodel.fit(x_train, y_train,
               epochs=best_epoch, 
               validation_split=VALIDATION_SPLIT, 
               callbacks=[stop_early])

eval_result = hypermodel.evaluate(x_test, y_test)
print("Eval result: ", eval_result)

# Save the model
hypermodel.save(f'{DIRECTORY}/{TUNED_MODEL}.h5')
# Save the best hyperparameters
with open(f'{DIRECTORY}/{TUNED_MODEL}.txt', 'w') as f:
    best_hps_dict = best_hps.values
    for key in best_hps_dict:
        f.write(key + ' ' + str(best_hps_dict[key]) + '\n')
    f.write('Best epoch: %d' % (best_epoch,))
    f.write('\n Eval result: ' + str(eval_result))
    f.write('\n')

# Save the history
import pickle
with open(f'{DIRECTORY}/{TUNED_MODEL}_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

# Save the tuner
tuner.results_summary()


Reloading Tuner from AmpDir/AmpHyperTune/tuner0.json
Train size:  59796
Test size:  538165
Best hyperparameters:
kernel_size_0 9
filters_0 32
activation_0 tanh
max_pooling True
pool_size 6
dropout_conv True
dropout_rate 0.4
units_dense0 352
activation_dense0 tanh
activation_output relu
learning_rate 0.0021299066766227647
tuner/epochs 2
tuner/initial_epoch 0
tuner/bracket 0
tuner/round 0
Epoch 1/10
1495/1495 [==============================] - 20s 13ms/step - loss: 0.6158 - accuracy: 0.4770 - val_loss: 0.5994 - val_accuracy: 0.4848
Epoch 2/10
1495/1495 [==============================] - 19s 13ms/step - loss: 0.6149 - accuracy: 0.4777 - val_loss: 0.5994 - val_accuracy: 0.4848
Epoch 3/10
1495/1495 [==============================] - 16s 11ms/step - loss: 0.6149 - accuracy: 0.4777 - val_loss: 0.5994 - val_accuracy: 0.4848
Epoch 4/10
1495/1495 [==============================] - 20s 14ms/step - loss: 0.6149 - accuracy: 0.4777 - val_loss: 0.5994 - val_accuracy: 0.4848
Epoch 5/10
1495/1495 [====

In [17]:
best_hps.values

{'kernel_size_0': 9,
 'filters_0': 32,
 'activation_0': 'tanh',
 'max_pooling': True,
 'pool_size': 6,
 'dropout_conv': True,
 'dropout_rate': 0.4,
 'units_dense0': 352,
 'activation_dense0': 'tanh',
 'activation_output': 'relu',
 'learning_rate': 0.0021299066766227647,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [ ]:
tuner.search(x_train, y_amp_train, epochs=50, validation_split=VALIDATION_SPLIT, callbacks=[stop_early])

In [29]:
def GetData(filename, train_frac=0.5, branches=['pileup', 'amp', 'phase', 'trace'],  treename="OutputTree"):
    '''
    Returns TFile as a pandas dataframe
    '''
    try: 
        import uproot
        import awkward as ak
        import numpy as np
        from sklearn.model_selection import train_test_split
        
    except ImportError:
        print("Please install uproot, awkward, numpy, and pandas")
        return
  
    file = uproot.open(filename)
    tree = file[treename]
    data = {}
    for branch in branches:
        data[branch] = ak.to_numpy(tree[branch].arrays()[branch])
    
    train_size = int(len(data["trace"])*train_frac)
    test_size = len(data["trace"]) - train_size
    print("Train size: ",train_size)
    print("Test size: ",test_size)

    x = np.array(data["trace"])
    y_pileup = np.array(data["pileup"])
    y_amp = np.array(data["amp"])
    y_phase = np.array(data["phase"])

    x_train, x_test, y_pileup_train, y_pileup_test, y_amp_train, y_amp_test, y_phase_train, y_phase_test = train_test_split(x, y_pileup, y_amp, y_phase, test_size=test_size)

    return ((x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test))

(x_train, y_pileup_train, y_amp_train, y_phase_train), (x_test, y_pileup_test, y_amp_test, y_phase_test) = GetData("../data/Data10percFloat.root")

Train size:  298980
Test size:  298981


In [ ]:

def PhaseNet(inputsize):
    model = keras.Sequential([
        layers.Conv1D(kernel_size=10, filters=64, activation='tanh', name='conv1', input_shape=(inputsize,1)),
        layers.Dropout(0.2, name='dropout'),
        layers.Conv1D(kernel_size=1, filters=64, activation='relu', name='conv2'),
        layers.Dropout(0.2, name='dropout'),
        layers.Conv1D(kernel_size=2, filters=64, activation='relu', padding='same', name='conv3'),
        layers.Dropout(0.2, name='dropout'),
        layers.Flatten(name='flat1'),
        layers.Dense(128, activation='relu', name='dense1'),
        layers.Dense(1, activation='linear', name='output')
    ],
    name='PhaseNet')

    return model

NUMEPOCHS = 50
BATCHSIZE = 32
PERCENTPILEUP = 0.5
ModelOutputName = 'tanner_test'
INPUTSIZE = TRACELENGTH-4*AUGMENTATION

model = PhaseNet(INPUTSIZE)

# early stopping callback based on validation loss
early_stopping = EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

model.compile(optimizer='adam',
                loss='mean_squared_error',
                metrics=['mean_squared_error'], 
                callbacks=[early_stopping])


x_train, x_test, y_train, y_test = du.GetDataSet(type="phase",
                                                 fname="DataSmallFloat.root",
                                                 tname="OutputTree")


model.fit(x_train, y_train, epochs=NUMEPOCHS, validation_split=0.2)

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)


_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)


#docs_infra: no_execute
%tensorboard --logdir={logdir}


model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)


print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))


converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Load MNIST dataset.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}

model = keras.Sequential([
    prune_low_magnitude(
        keras.layers.Conv2D(
            32, 5, padding='same', activation='relu',
            input_shape=(28, 28, 1),
            name="pruning_sparsity_0_5"),
        **pruning_params_sparsity_0_5),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    prune_low_magnitude(
        keras.layers.Conv2D(
            64, 5, padding='same',
            name="structural_pruning"),
        **pruning_params_2_by_4),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    keras.layers.Flatten(),
    prune_low_magnitude(
        keras.layers.Dense(
            1024, activation='relu',
            name="structural_pruning_dense"),
        **pruning_params_2_by_4),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

batch_size = 128
epochs = 2

model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    callbacks=tfmot.sparsity.keras.UpdatePruningStep(),
    validation_split=0.1)

_, pruned_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Pruned test accuracy:', pruned_model_accuracy)

model = tfmot.sparsity.keras.strip_pruning(model)

import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

  # Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file, experimental_preserve_all_tensors=True)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

print(f"Shape of Dense layer is {tensor_data.shape}")

import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24

subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)


def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')

        plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
op_details = interpreter._get_ops_details()
op_name = 'CONV_2D'
op_detail = [x for x in op_details if op_name in x["op_name"]]
tensor_data = interpreter.tensor(op_detail[1]["inputs"][1])()
print(f"Shape of the weight tensor is {tensor_data.shape}")


weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()

# Get weights of the convolutional layer that has been pruned with random pruning.
tensor_name = 'pruning_sparsity_0_5/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[0]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

In [3]:
import tempfile
import os


import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:

def GetPlot(model_results, features):
        ymin = 1
        ymax = 20000
        layerX = [np.ones(len(features))]
        layerY = [np.linspace(ymin,ymax,len(features))]
        layer_probabilities = [np.ones(len(features))]
        layer_weights = []
        for i in range(len(model_results['activation_prob'])):
                layer_probabilities.append(model_results['activation_prob'][i])
                layer_weights.append(model_results['weights'][i])
                layerX.append(np.ones(len(model_results['activation_prob'][i]))*(i+2))
                if i == len(model_results['activation_prob'])-1:
                        layerY.append(np.array([(ymax-ymin)/2]))
                else:
                        layerY.append(np.linspace(ymin,ymax,len(model_results['activation_prob'][i])))

        line_iXY = []
        line_fXY = []
        line_W = []
        node_XY = []
        node_W = []

        cutoff = 0.0
        for iW, w in enumerate(layer_weights):
                for iY, iprob in enumerate(layer_probabilities[iW]):
                        if iprob > cutoff:
                                node_XY.append([layerX[iW][iY],layerY[iW][iY]])
                                node_W.append(layer_probabilities[iW][iY])
                                for jY, jprob in enumerate(layer_probabilities[iW+1]):
                                        if jprob > cutoff:
                                                node_XY.append([layerX[iW+1][jY],layerY[iW+1][jY]])
                                                node_W.append(layer_probabilities[iW+1][jY])
                                                line_iXY.append([layerX[iW][iY],layerY[iW][iY]])
                                                line_fXY.append([layerX[iW+1][jY],layerY[iW+1][jY]])
                                                line_W.append(np.abs(w[iY,jY]/np.max(np.abs(w[:,jY]))))    
        line_iXY = np.array(line_iXY)
        line_fXY = np.array(line_fXY)
        line_W = np.array(line_W)
        node_XY = np.array(node_XY)
        node_W = np.array(node_W)
        fig = plt.figure(figsize=(12,6))
        ax = fig.add_subplot(111)   
        for A, B, W in zip(line_iXY, line_fXY, line_W):
                ax.plot([A[0]+0.03, B[0]-0.03], [A[1], B[1]], c=cm.Blues(W), linewidth=0.5, alpha=0.5, linestyle='-', marker=' ', markersize=0)
        ax.scatter(node_XY[:,0],node_XY[:,1],c=node_W,cmap=cm.Reds,s=30,alpha=1.0)
        # plt.show()
        
        Plot = {}
        Plot['iXY'] = line_iXY
        Plot['fXY'] = line_fXY
        Plot['lineW'] = line_W
        Plot['nodeXY'] = node_XY
        Plot['nodeW'] = node_W
        
        return Plot

def PruneModel(model,pdf_file, features, target):


        # probability cutoff
        pdf =  pd.read_hdf(pdf_file, key='df').sample(frac=0.5).copy()
        cutoff = 0.0
        model.summary()
        model.compile(loss='mse', optimizer='adam', metrics=['mse'])
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        model.fit(pdf[features].values, pdf[target].values, epochs=100, batch_size=128, verbose=0, validation_split=0.2, callbacks=[es])
        layer_in_model = []
        for i in range(len(model.layers)):
                layer_in_model.append(model.get_layer(index=i))

        X = pdf[features].values 
        layer_activation_probabilities = {}
        layer_weights = {}
        layer_biases = {}
        weights_for_plot = []
        for i in range(len(layer_in_model)):
                
                layer_outputs = layer_in_model[i](X)
                layer_activation_probabilities[i] = np.count_nonzero(np.array(layer_outputs), axis=0)/X.shape[0]
                layer_weights[i] = layer_in_model[i].get_weights()[0]
                layer_biases[i] = layer_in_model[i].get_weights()[1]
                weights_for_plot.append(layer_weights[i])
                for j in range(layer_activation_probabilities[i].shape[0]):
                        if layer_activation_probabilities[i][j] <= cutoff:
                                        layer_weights[i][:,j] = 0.0
                                        layer_biases[i][j] = 0.0
                X = layer_in_model[i](X)   
                
        for i in range(1,len(layer_in_model)):
                for j in range(layer_activation_probabilities[i-1].shape[0]):
                        if layer_activation_probabilities[i-1][j] <= cutoff:
                                        layer_weights[i][j,:] = 0.0                               

        new_weights = {}
        new_biases = {}
        for i in range(1,len(layer_in_model)):
                weights = []
                bias = []
                for j in range(layer_weights[i].shape[0]):
                        w = layer_weights[i][j,:]
                        w = w[w != 0.0]
                        if w.size != 0:
                                weights.append(w)
                for j in range(layer_biases[i].shape[0]):
                        b = layer_biases[i][j]
                        if b != 0.0:
                                bias.append(b)
                                
                new_weights[i] = np.array(weights)
                new_biases[i] = np.array(bias)
                
        active_nodes = []
        for i in range(len(layer_in_model)):
                if np.count_nonzero(layer_activation_probabilities[i])  == 0:
                        active_nodes.append(1)
                else: 
                        active_nodes.append(np.count_nonzero(layer_activation_probabilities[i]))

        weights_for_new_model = []
        bias_for_new_model = []
        for w in new_weights:
                weights_for_new_model.append(new_weights[w])
                bias_for_new_model.append(new_biases[w])

        weights_for_plot = []
        for w in layer_weights:
                weights_for_plot.append(layer_weights[w])

        nodes_for_new_model = active_nodes 
        layer_activation_probabilities_for_plot = []
        for i in range(len(layer_in_model)):
                layer_activation_probabilities_for_plot.append(layer_activation_probabilities[i])
                
        mse_of_model = model.evaluate(pdf[features], pdf[target], verbose=0)[1]
        print('MSE of model: ', mse_of_model)   

        reduced_sum = 0
        for i in range(1,len(layer_in_model)):
                w2 = tf.reduce_sum(tf.square(layer_weights[i]))
                reduced_sum += w2.numpy()
        print('Reduced sum of W^2: ', reduced_sum)


        model_results = {}
        model_results['mse'] = mse_of_model
        model_results['activation_prob'] = layer_activation_probabilities_for_plot
        model_results['weights'] = weights_for_plot
        model_results['reduced_sum'] = reduced_sum
        
        new_model = {}
        new_model['weights'] = weights_for_new_model
        new_model['bias'] = bias_for_new_model
        new_model['nodes'] = nodes_for_new_model
        
        return model_results, new_model

def GetDNNPlot(kernel_reg = False, lam=0.0):
                
        filename = "/lustre/isaac/scratch/tmengel/jet-background-subtraction/datasets/root-files/AuAu_R04/test/AuAu_R04_50_test_sample.h5"
        dnn_features = ["jet_pt_raw","jet_nparts","jet_area","jet_angularity","jet_track_pt_0",
                                "jet_track_pt_1","jet_track_pt_2","jet_track_pt_3","jet_track_pt_4",
                                "jet_track_pt_5","jet_track_pt_6","jet_track_pt_7"]
        
        # dnn_features = ["jet_pt_raw","jet_nparts","jet_area","jet_angularity","jet_track_pt_0"]
        

        target='jet_pt_truth'
                        
        if kernel_reg:           
                model = keras.Sequential([
                tf.keras.layers.Dense(100,input_shape=(len(dnn_features),),activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam)),
                tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam)),
                tf.keras.layers.Dense(50,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam)),
                tf.keras.layers.Dense(1,activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam))
                ]
                )
        else:
                model = keras.Sequential([
                tf.keras.layers.Dense(100,input_shape=(len(dnn_features),),activation='relu',kernel_initializer='he_uniform'),
                tf.keras.layers.Dense(100,activation='relu',kernel_initializer='he_uniform'),
                tf.keras.layers.Dense(50,activation='relu',kernel_initializer='he_uniform'),
                tf.keras.layers.Dense(1,activation='relu',kernel_initializer='he_uniform')
                ])
                
                
        model_results, new_model = PruneModel(model,filename,features=dnn_features,target=target)
        PLOTS = GetPlot(model_results, dnn_features)
        trainable = 0
        new_nodes = new_model['nodes']
        print(new_nodes)
        new_nodes = np.array(new_nodes)
        trainable = np.sum(new_nodes)
        PLOTS['trainable'] = trainable
        PLOTS['new_nodes'] = new_nodes
        PLOTS['mse'] = model_results['mse']
        PLOTS['lam'] = lam
        PLOTS['reduced_sum'] = model_results['reduced_sum']
        
        return PLOTS

def GetNNPlot(architechture, lam=0.0):
        
        filename = "/lustre/isaac/scratch/tmengel/jet-background-subtraction/datasets/root-files/AuAu_R04/test/AuAu_R04_50_test_sample.h5"
        # dnn_features = ["jet_pt_raw","jet_nparts","jet_area","jet_angularity","jet_track_pt_0",
        #                         "jet_track_pt_1","jet_track_pt_2","jet_track_pt_3","jet_track_pt_4",
        #                         "jet_track_pt_5","jet_track_pt_6","jet_track_pt_7"]
        
        dnn_features = ["jet_pt_raw","jet_nparts","jet_area","jet_angularity","jet_track_pt_0"]

        target='jet_pt_truth'
        layers_list = []
        # input layer    
        layers_list.append(layers.Dense(architechture[0],input_shape=(len(dnn_features),),activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam)))
        # hidden layersl
        for i in range(1,len(architechture)-1):
                layers_list.append(layers.Dense(architechture[i],activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam)))
        # output layer
        layers_list.append(layers.Dense(architechture[-1],activation='relu',kernel_initializer='he_uniform',kernel_regularizer=keras.regularizers.l2(lam)))

        model = keras.Sequential(layers_list)

        model_results, new_model = PruneModel(model,filename,features=dnn_features,target=target)
        PLOTS = GetPlot(model_results, dnn_features)
        trainable = 0
        new_nodes = new_model['nodes']
        # print(new_nodes)
        new_nodes = np.array(new_nodes)
        trainable = np.sum(new_nodes)
        PLOTS['trainable'] = trainable
        PLOTS['new_nodes'] = new_nodes
        PLOTS['mse'] = model_results['mse']
        PLOTS['lam'] = lam
        PLOTS['reduced_sum'] = model_results['reduced_sum']
        
        return PLOTS

def CombineLayersBelowNodeThreshold(node_list, threshold):
        # combine layers below a certain threshold to the previous layer
        new_node_list = []
        # loop in reverse order
        for i in range(len(node_list)-2,-1,-1):
                if node_list[i] < threshold:
                        node_list[i-1] = node_list[i-1] + node_list[i]
                        node_list[i] = 0
        for i in range(len(node_list)):
                if node_list[i] != 0:
                        new_node_list.append(node_list[i])
        return new_node_list


def MinimizeArchitecture(mse_threshold=0.05):
        NETWORKS = {}
        NETWORKS['initial'] = GetDNNPlot(kernel_reg=True, lam=0.01)
        
        original_mse = NETWORKS['initial']['mse']
        reduced_sum = NETWORKS['initial']['reduced_sum']
        new_nodes = NETWORKS['initial']['new_nodes']
        new_nodes_trimmed = CombineLayersBelowNodeThreshold(new_nodes, 3)

        mse = original_mse
        prunning_iteration = 0
        while True:
                percent_change = (mse - original_mse)/original_mse
                lam = (mse_threshold*original_mse)/reduced_sum
                new_nodes_trimmed = CombineLayersBelowNodeThreshold(new_nodes, 3)
                
                if len(new_nodes_trimmed) == 1:
                        print('Only one layer left')
                        break
                
                print(f'Prunning iteration: {prunning_iteration}')
                print(f'MSE: {mse}, Original MSE: {original_mse}, Percent Change: {percent_change*100}%')
               
                if percent_change > mse_threshold:
                        print('Percent change is greater than threshold')
                        break
                
                else:
                        print(f'Lambda: {lam}')
                        print(f'New Nodes: {new_nodes}')
                        print(f'New Nodes Trimmed: {new_nodes_trimmed}') 
                                               
                        TEMP_NETWORK = GetNNPlot(new_nodes_trimmed, lam)
                        mse_tmp = TEMP_NETWORK['mse']
                        if mse_tmp > 100: # error try again
                                continue
                        prunning_iteration += 1 
                        NETWORKS[f'iter_{prunning_iteration}'] = TEMP_NETWORK
                        mse = NETWORKS[f'iter_{prunning_iteration}']['mse']
                        new_nodes = NETWORKS[f'iter_{prunning_iteration}']['new_nodes']
                        reduced_sum = NETWORKS[f'iter_{prunning_iteration}']['reduced_sum']

        return NETWORKS

In [2]:
NUMEPOCHS = 100
PHASEMAX = 100
PERCENTPILEUP = 0.5
NUMTRAINING = 20000
ModelOutputName = 'tanner_test'
AUGMENTATION = 8
TRACELENGTH = 250
INPUTSIZE = TRACELENGTH-4*AUGMENTATION

In [10]:
def PileupNet(inputSize, 
                kernel_size = 4,
                filters = 16,
                dense_units = 32,
                verbose=True,
                kernel_reg = False,
                lambda_reg = 0.000
                ):


    if kernel_reg:
         model = keras.Sequential(
              [
                    tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='conv1'),
                    tf.keras.layers.Flatten(name='flatten'),
                    tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='dense'),
                    tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='output')
                ],
                name='PileupNet'
            )
    else:
        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', name='conv1'),
                tf.keras.layers.Flatten(name='flatten'),
                tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', name='dense'),
                tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_uniform', name='output')
            ],
            name='PileupNet'
        )

    if verbose:
        model.summary()
    return model

def AmpNet(inputSize, 
                kernel_size = 10,
                filters = 16,
                dense_units = 128,
                verbose=True,
                kernel_reg = False,
                lambda_reg = 0.000
                ):


    if kernel_reg:
         model = keras.Sequential(
              [
                    tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='tanh', input_shape=(inputSize,1), kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='conv1'),
                    tf.keras.layers.Flatten(name='flatten'),
                    tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='dense'),
                    tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='output')
                ],
                name='AmpNet'
            )
    else:
        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='tanh', input_shape=(inputSize,1), kernel_initializer='he_uniform', name='conv1'),
                tf.keras.layers.Flatten(name='flatten'),
                tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', name='dense'),
                tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', name='output')
            ],
            name='AmpNet'
        )

    if verbose:
        model.summary()
    return model

def PhaseNet(inputSize, 
                kernel_size = 10,
                filters = 16,
                dense_units = 128,
                verbose=True,
                kernel_reg = False,
                lambda_reg = 0.000
            ):


    if kernel_reg:
         model = keras.Sequential(
              [
                    tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='conv1'),
                    tf.keras.layers.Flatten(name='flatten'),
                    tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='dense'),
                    tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(lambda_reg), name='output')
                ],
                name='PhaseNet'
            )
    else:
        model = keras.Sequential(
            [
                tf.keras.layers.Conv1D(kernel_size=kernel_size, filters=filters, activation='relu', input_shape=(inputSize,1), kernel_initializer='he_uniform', name='conv1'),
                tf.keras.layers.Flatten(name='flatten'),
                tf.keras.layers.Dense(dense_units, activation='relu', kernel_initializer='he_uniform', name='dense'),
                tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_uniform', name='output')
            ],
            name='PhaseNet'
        )

    if verbose:
        model.summary()
    return model


In [11]:
trace_train, trace_test, pileup_train, pileup_test = GetDataSet(type="phase")
model = TraceNet()
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# # early stopping
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# # model checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(ModelOutputName+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(trace_train, pileup_train, epochs=NUMEPOCHS, batch_size=64, validation_split=0.5, callbacks=[es,mc])

Train size:  430532
Test size:  107633
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 218, 1)]          0         
                                                                 
 conv1 (Conv1D)              (None, 215, 16)           80        
                                                                 
 flatten1 (Flatten)          (None, 3440)              0         
                                                                 
 dense1 (Dense)              (None, 32)                110112    
                                                                 
 pileupoutput (Dense)        (None, 1)                 33        
                                                                 
Total params: 110,225
Trainable params: 110,225
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
33

KeyboardInterrupt: 